<a href="https://colab.research.google.com/github/Hpkarimi/googleColab/blob/main/projectDb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import time
import pandas as pd
from pandas_datareader.compat import StringIO
from bs4 import BeautifulSoup as bs
import numpy as np

In [4]:
#response = requests.get('http://www.tsetmc.com/tsev2/data/MarketWatchInit.aspx?h=0&r=0', timeout=15)
#data = pd.read_csv(StringIO(response.text), sep='@', header=None)
 
response = pd.read_csv('./drive/MyDrive/MarketWatchInit(1).aspx', sep='@', header=None)
response

,0,1,2,3,4
0,"103225,667624,397879","99/12/20 16:50:26,F,1206878.02,<div class='mn'...","109297506785423,IRO9PTEH6021,ضترا3001,اختيارخ ...","109297506785423,1,1,1,1555,2750,50,6;114312662...",7940239860


In [5]:
dataIdx = pd.read_csv(StringIO(response.loc[0, 1]) , header=None
          , index_col=0
          , names=['lastTradeDTPersian', 'statusBors', 'idxBors', 'idxBorsChangeP',
                                                 'valBazarBors', 'volBors', 'valBors', 'cntBors', 'statusFrabors',
                                                 'volFrabors', 'valFrabors', 'cntFrabors', 'statusMoshtagheh',
                                                  'volMoshtagheh', 'valMoshtagheh', 'cntMoshtagheh','na']
                                          #, parse_dates={'lastTradeDT': [0]}
#                                          # , date_parser=(
 #                           lambda col: JalaliDatetime.strptime(col, '%y/%m/%d %H:%M:%S').todatetime())
                                          , keep_date_col=True )
dataIdx

,statusBors,idxBors,idxBorsChangeP,valBazarBors,volBors,valBors,cntBors,statusFrabors,volFrabors,valFrabors,cntFrabors,statusMoshtagheh,volMoshtagheh,valMoshtagheh,cntMoshtagheh,na
lastTradeDTPersian,,,,,,,,,,,,,,,,
99/12/20 16:50:26,F,1206878.02,<div class='mn'>(3268.61)</div> -0.27%,4.821440e+16,4.380217e+09,5.226380e+13,527411,F,950384182.0,1.938748e+14,176346,F,787132.0,2.844773e+10,2328,NaN


In [9]:
dataInv = pd.read_csv(StringIO(response.loc[0, 2])
                                          , header=None
                                          , lineterminator=';'
                                          , index_col=0
                                          ,
                                          names=['id', 'isin', 'namad', 'name', 'lastTime', 'open', 'close', 'last', 'cnt',
                                                 'volume',
                                                 'value', 'low', 'high', 'yClose', 'eps', 'mabna', 'na1', 'bazar',
                                                 'groupCode',
                                                 'dayLimitHigh', 'dayLimitLow', 'share', 'typeNamad']
                                          )
dataInv

,isin,namad,name,lastTime,open,close,last,cnt,volume,value,low,high,yClose,eps,mabna,na1,bazar,groupCode,dayLimitHigh,dayLimitLow,share,typeNamad
id,,,,,,,,,,,,,,,,,,,,,,
109297506785423,IRO9PTEH6021,ضترا3001,اختيارخ شتران-7000-1400/03/09,63709,0,2944,3000,0,0,0,0,0,2944,NaN,1,464,3,23,100000.0,1.0,1000,311
114312662654155,IRO3SAIZ0001,وسپهر,سرمايه گذاري مالي سپهرصادرات,122957,10100,10100,10101,9026,21154023,213659214914,10100,10102,10100,1236.0,11881189,6491,2,56,10706.0,9898.0,30000000000,303
143187001116603,IRO6MELZ96C1,تملي612,تسهيلات مسكن بانك ملي-اسفند96,164843,935608,891056,870122,0,0,0,0,0,891056,NaN,1,515,2,59,935608.0,846504.0,5870,307
268288798122357,IRK1A0520081,زعف0008پ15,زعفران0008پوشالمعمول طلاسرخ(پ),104722,81500,81500,81500,1,100,8150000,81500,81500,83236,NaN,1,622,7,42,85723.0,77559.0,500000,701
318005355896147,IRO7PFSP0001,فسا,پتروشيمي فسا,122235,1810,1846,1810,7,27851,50410310,1810,1810,1846,7.0,10834237,2412,4,44,1901.0,1810.0,3486000000,309
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71787506425833715,IRO9PASN4831,ضسان2006,اختيارخ پارسان-12750-00/02/15,63709,0,5500,5500,0,0,0,0,0,5500,NaN,1,474,3,44,100000.0,1.0,1000,311
71856634742001725,IRO1SEPK0001,سپ,پرداخت الكترونيك سامان كيش,122648,16760,17070,16760,121,293834,4926118680,16760,17080,17100,511.0,2929115,3160,1,72,18120.0,16760.0,5000000000,300
71901111301490182,IRB3TB490051,اخزا718,اسنادخزانه-م18بودجه97-000525,120241,921000,919886,921000,25,106454,97925577467,916176,921000,920706,NaN,1,1132,2,69,966741.0,874671.0,24598918,306


In [10]:
namad = dataInv.copy()
namad = namad.drop(['lastTime','open','close','last','cnt','volume','value','low','high','yClose','dayLimitHigh','dayLimitLow']
                   , axis=1)
namad

,isin,namad,name,eps,mabna,na1,bazar,groupCode,share,typeNamad
id,,,,,,,,,,
109297506785423,IRO9PTEH6021,ضترا3001,اختيارخ شتران-7000-1400/03/09,NaN,1,464,3,23,1000,311
114312662654155,IRO3SAIZ0001,وسپهر,سرمايه گذاري مالي سپهرصادرات,1236.0,11881189,6491,2,56,30000000000,303
143187001116603,IRO6MELZ96C1,تملي612,تسهيلات مسكن بانك ملي-اسفند96,NaN,1,515,2,59,5870,307
268288798122357,IRK1A0520081,زعف0008پ15,زعفران0008پوشالمعمول طلاسرخ(پ),NaN,1,622,7,42,500000,701
318005355896147,IRO7PFSP0001,فسا,پتروشيمي فسا,7.0,10834237,2412,4,44,3486000000,309
...,...,...,...,...,...,...,...,...,...,...
71787506425833715,IRO9PASN4831,ضسان2006,اختيارخ پارسان-12750-00/02/15,NaN,1,474,3,44,1000,311
71856634742001725,IRO1SEPK0001,سپ,پرداخت الكترونيك سامان كيش,511.0,2929115,3160,1,72,5000000000,300
71901111301490182,IRB3TB490051,اخزا718,اسنادخزانه-م18بودجه97-000525,NaN,1,1132,2,69,24598918,306


In [11]:
dataInv.drop(columns=['isin','name','mabna','bazar','groupCode','share','typeNamad'], inplace=True)#'namad','eps',
dataInv

,namad,lastTime,open,close,last,cnt,volume,value,low,high,yClose,eps,na1,dayLimitHigh,dayLimitLow
id,,,,,,,,,,,,,,,
109297506785423,ضترا3001,63709,0,2944,3000,0,0,0,0,0,2944,NaN,464,100000.0,1.0
114312662654155,وسپهر,122957,10100,10100,10101,9026,21154023,213659214914,10100,10102,10100,1236.0,6491,10706.0,9898.0
143187001116603,تملي612,164843,935608,891056,870122,0,0,0,0,0,891056,NaN,515,935608.0,846504.0
268288798122357,زعف0008پ15,104722,81500,81500,81500,1,100,8150000,81500,81500,83236,NaN,622,85723.0,77559.0
318005355896147,فسا,122235,1810,1846,1810,7,27851,50410310,1810,1810,1846,7.0,2412,1901.0,1810.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71787506425833715,ضسان2006,63709,0,5500,5500,0,0,0,0,0,5500,NaN,474,100000.0,1.0
71856634742001725,سپ,122648,16760,17070,16760,121,293834,4926118680,16760,17080,17100,511.0,3160,18120.0,16760.0
71901111301490182,اخزا718,120241,921000,919886,921000,25,106454,97925577467,916176,921000,920706,NaN,1132,966741.0,874671.0


In [24]:
pe = dataInv['close'].values / dataInv['eps'].fillna(9999999)
pe.where(pe<4, inplace=True)
pe.where(pe>2, inplace=True)
pe.dropna(inplace=True)
pe_namad = namad.merge(pe, left_index=True, right_index=True)
pe_namad

,isin,namad,name,eps_x,mabna,na1,bazar,groupCode,share,typeNamad,eps_y
id,,,,,,,,,,,
11258722998911897,IRO1SKBV0001,وسكاب,س.ص.بازنشستگي كاركنان بانكها,2254.0,9505703,3903,1,56,6000000000,300,2.333629
11358107932902023,IRO1OS260001,وسمازن,س.سهام عدالت استان مازندران,394.0,75428649,1175,1,56,188571621779,300,2.461929
15039949673085566,IRO1AYEG0001,پرديس,سرمايه گذاري پرديس,1569.0,9157509,3793,1,56,1250000000,300,3.479924
17528249960294496,IRO1TMEL0001,وتوسم,سرمايه‌گذاري‌توسعه‌ملي‌,3093.0,4990020,4348,1,56,4500000000,300,3.239573
19310456400689867,IRO7IEIP0001,خبازرس,بازرسي مهندسي و صنعتي ايران,13147.0,553588,1642,4,71,150000000,309,2.748003
24212636157410845,IRO7ARNP0001,وآرين,شركت توسعه اقتصادي آرين,1121.0,5151984,2927,4,56,2380000000,309,3.462979
28328710198554144,IRO1BALI0001,وبوعلي,سرمايه‌گذاري‌بوعلي‌,2042.0,8787346,3631,1,56,3000000000,300,2.786484
28809886765682162,IRO1HSHM0001,حفاري,حفاري شمال,1935.0,7012623,5723,1,11,9000000000,300,3.684755
31049085025064185,IRO7THSP0001,شتهران,داروسازي تهران شيمي,2186.0,1298196,2121,4,43,445500000,309,3.523788


In [ ]:
dataInvSaf = pd.read_csv(StringIO(response.loc[0, 3])
                                                 , header=None
                                                 , lineterminator=';'
                                                 , index_col=0
                                                 , names=['id', 'row', 'cntSell', 'cntBuy', 'prcBuy', 'prcSell', 'volBuy',
                                                          'volSell'])
dataInvSaf

,row,cntSell,cntBuy,prcBuy,prcSell,volBuy,volSell
id,,,,,,,
109297506785423,1,1,1,1555,2750,50,6
114312662654155,1,2,1476,10100,10100,282479591,2192
143187001116603,1,0,2,935608,0,12,0
268288798122357,1,2,1,81009,84500,63,200
318005355896147,1,249,0,0,1810,0,7999744
...,...,...,...,...,...,...,...
71758511001096824,3,2,0,0,67755,0,592
71856634742001725,3,6,0,0,17000,0,53270
71901111301490182,3,1,1,918652,962664,100,337


In [40]:
# اطلاعات تاریخی سهام عادی و تعدیل شده
# https://members.tsetmc.com/tsev2/chart/data/Financial.aspx?i=69143674941561637&t=ph&a=0
# https://members.tsetmc.com/tsev2/chart/data/Financial.aspx?i=69143674941561637&t=ph&a=1 تعدیل شده
historyDataCSV = '''20160425,1109,1071,1103,1103,7455098,1100;20160426,1126,1074,1126,1102,14544283,1101;20160427,1109,1060,1105,1073,5582589,1079;20160430,1095,1060,1063,1067,7180996,1072;20160501,1072,1053,1072,1053,5008436,1065;20160502,1067,1031,1060,1032,8022211,1051;20160503,1049,1024,1049,1026,6052528,1031;20160504,1043,1012,1028,1017,5836289,1022;20160507,1026,990,1026,997,6856292,1005;20160508,1055,994,1004,1055,15816615,1045;20160509,1081,1031,1069,1045,6104604,1057;20160510,1067,1010,1050,1024,2166717,1047;20160511,1037,996,1037,1010,4194469,1021;20160514,1035,996,1000,1011,3592224,1019;20160515,1024,989,1024,1000,8250561,995;20160516,1016,992,992,1002,4180954,999;20160517,1023,996,1015,1006,4758992,1005;20160518,1017,997,1016,1005,4158518,1004;20160521,1014,996,1014,998,5694761,1000;20160523,1014,992,997,996,4392636,1000;20160524,1009,988,1009,990,6238033,993;20160525,1006,982,990,995,5838420,994;20160528,1005,986,1003,989,2688666,993;20160529,1002,983,1002,985,3175926,991;20160530,1003,979,992,984,10529601,997;20160531,1000,977,992,978,2762451,991;20160601,988,971,978,975,4447015,980;20160605,1003,969,969,973,3249271,983;20160606,992,975,992,978,1366515,983;20160607,983,961,976,963,3622297,976;20160608,995,968,968,989,11744164,985;20160611,982,952,976,953,3121547,975;20160612,974,944,974,954,5357488,959;20160613,967,911,964,919,9495775,931;20160614,947,895,918,926,7699181,924;20160615,945,913,925,929,2742701,927;20160618,944,898,932,904,1352672,924;20160619,911,889,904,895,1600172,918;20160620,918,872,895,880,6839435,884;20160621,908,874,880,891,3129574,888;20160622,932,910,913,918,6744969,913;20160625,957,911,953,945,9661146,946;20160626,992,927,942,975,66226950,990;20160628,1003,968,996,978,20814605,990;20160629,988,954,961,975,7526953,970;20160702,1007,958,987,997,17928473,988;20160703,1020,994,998,1016,27542587,1002;20160704,1035,992,1016,1019,48747064,1009;20160705,1010,964,998,989,54384684,990;20160709,980,954,975,959,6933929,970;20160710,968,929,951,959,3706752,959;20160711,989,940,964,973,2390180,964;20160712,974,951,955,966,3497481,962;20160713,974,945,966,963,5124030,961;20160716,972,954,972,957,7342012,962;20160717,975,950,957,951,3522387,962;20160718,961,941,954,961,1928466,959;20160719,964,943,943,954,5850227,955;20160720,1002,954,961,1002,9535325,983;20160723,1031,1002,1002,1031,48068808,1031;20160724,1038,1009,1038,1012,3354714,1026;20160725,1022,995,1010,1002,6722644,1010;20160726,1010,988,997,992,14669824,991;20160727,1040,980,985,1040,45248463,1033;20160731,1067,1015,1040,1015,5047315,1039;20160801,1042,1019,1025,1026,5584945,1029;20160802,1056,1012,1013,1053,11154650,1033;20160803,1056,987,1042,995,30916528,1015;20160806,990,965,990,974,18755728,973;20160807,982,939,959,944,20472818,957;20160808,961,939,947,954,5347081,947;20160809,959,942,954,943,2723470,949;20160810,954,936,947,937,3773804,944;20160813,951,935,949,937,3430396,942;20160814,946,929,939,930,5028667,937;20160815,951,922,951,931,3581099,934;20160816,934,918,933,925,6363271,923;20160817,947,920,947,924,1914412,923;20160820,939,917,929,932,3534827,924;20160821,944,920,920,934,2185789,926;20160822,942,919,942,931,4028294,927;20160823,938,915,938,917,3190339,924;20160824,932,910,917,918,6465304,920;20160827,922,898,913,898,4073676,915;20160828,918,886,918,887,5817401,899;20160829,943,923,943,924,19447949,939;20160830,941,925,939,935,4488197,937;20160831,962,925,925,932,7528665,946;20160903,939,906,929,920,7602645,925;20160904,928,905,908,907,4598340,920;20160905,916,904,904,908,1218158,918;20160906,918,885,915,904,11412317,900;20160924,949,915,948,922,4110215,934;20160925,937,900,935,906,6293229,914;20160926,914,893,897,901,2627747,909;20160927,908,890,906,900,4688470,902;20160928,935,896,901,915,10950986,916;20161001,933,901,932,903,6994772,913;20161002,915,883,915,893,7373649,893;20161003,893,869,893,871,10255060,876;20161004,875,854,857,872,11726402,860;20161005,888,860,865,867,7976110,869;20161008,901,869,901,875,8453329,883;20161009,886,863,878,865,9517086,866;20161010,879,864,872,872,9777586,870;20161015,887,862,873,871,8202409,867;20161016,872,857,869,861,5643139,863;20161017,906,857,860,906,17525415,898;20161018,943,916,923,922,59998327,936;20161019,943,906,928,906,11159276,922;20161022,914,885,906,886,11426873,893;20161023,901,870,901,880,8237621,879;20161024,886,865,883,877,8543799,878;20161025,899,878,879,883,8692117,883;20161026,896,872,879,879,10393302,880;20161029,924,886,886,924,19899709,922;20161030,948,909,936,917,41839260,928;20161031,933,908,933,908,10708684,915;20161101,922,883,922,896,6668571,902;20161102,908,886,907,893,3549819,898;20161105,901,875,885,878,4403017,890;20161106,890,877,877,886,2352545,888;20161107,908,888,896,907,3831239,893;20161108,911,883,901,891,3459608,893;20161109,879,849,858,878,14394356,859;20161112,886,858,878,863,9519699,866;20161113,879,852,875,852,35103271,862;20161114,868,852,862,856,12466793,858;20161115,893,862,872,872,11801425,879;20161116,909,873,882,886,111452535,907;20161119,893,878,893,883,4270929,893;20161121,886,870,886,875,2552396,888;20161122,886,868,872,878,4181995,882;20161123,883,868,875,872,2180891,879;20161126,878,868,870,875,3478227,876;20161127,878,864,872,867,3658019,872;20161129,904,865,866,901,20224762,894;20161203,915,879,911,880,9326932,891;20161204,889,850,886,871,9399062,865;20161205,882,865,871,873,6883328,871;20161206,900,873,873,896,9267297,893;20161207,895,872,895,886,9528194,888;20161210,892,876,888,882,2814144,887;20161211,881,865,872,873,2814512,881;20161212,878,867,873,867,3942876,874;20161213,885,864,878,864,27055129,867;20161214,872,857,872,860,6197478,864;20161218,865,855,865,860,4216186,861;20161219,870,851,857,851,2661122,859;20161220,856,845,856,847,3657232,855;20161221,855,844,847,848,3863272,851;20161224,852,839,852,839,6935421,844;20161227,865,844,865,847,2571892,847;20161228,853,836,853,843,1210609,847;20161231,849,829,848,829,6318346,837;20170101,823,807,822,821,9291303,813;20170102,820,800,815,802,2236582,811;20170103,812,798,812,799,3015717,808;20170104,811,795,797,796,1223446,807;20170107,800,784,797,785,4313662,798;20170108,793,771,791,782,4481601,786;20170109,792,768,792,772,3037092,781;20170111,779,764,771,779,1881948,779;20170114,795,778,780,785,1821517,781;20170115,785,770,780,779,3020723,779;20170116,790,768,778,768,1849688,778;20170117,782,764,767,771,4037889,775;20170118,780,768,771,774,2694934,773;20170121,778,766,777,770,2439786,772;20170122,774,764,774,768,2180049,771;20170123,785,763,764,785,4281428,772;20170124,793,776,776,787,2401164,777;20170125,797,782,784,785,1630503,779;20170128,782,767,782,770,1507952,777;20170129,775,767,771,772,2169743,775;20170130,782,770,770,771,4667165,775;20170131,813,768,771,813,15844641,798;20170201,819,787,819,789,2599106,798;20170204,793,772,793,778,3539972,788;20170205,778,764,775,778,4370337,777;20170206,782,762,766,763,4669967,770;20170207,769,761,769,764,6677551,766;20170208,767,756,764,761,5137492,762;20170211,763,749,759,749,3727387,759;20170212,757,739,757,757,5792726,752;20170213,760,741,757,742,2393604,752;20170214,750,737,742,743,3809271,746;20170215,749,739,741,748,5653956,745;20170218,756,741,741,753,4584038,747;20170219,759,750,757,756,19637934,757;20170220,763,749,760,752,2727958,756;20170221,775,749,749,764,4301845,758;20170222,775,754,762,757,7400970,764;20170225,780,764,764,770,9608701,770;20170226,787,764,775,764,8016190,775;20170227,777,751,777,760,2385903,770;20170228,760,746,750,749,4856060,757;20170301,752,742,744,749,3872831,752;20170304,746,733,743,734,7030800,739;20170305,739,723,732,738,5791594,733;20170306,770,730,738,749,5603165,746;20170307,748,735,748,737,1453151,745;20170308,739,730,739,731,2159831,741;20170311,743,721,743,725,5248468,732;20170312,738,722,726,726,3899701,729;20170313,731,721,726,722,5814950,726;20170314,742,722,726,729,1640987,727;20170315,746,729,735,732,1516175,728;20170318,742,726,726,740,1033823,729;20170325,742,728,742,732,2268915,730;20170326,752,731,734,741,4259110,733;20170327,757,733,733,757,3160371,740;20170328,777,744,757,777,5757038,764;20170329,795,767,782,785,6388665,781;20170403,806,780,793,805,12670873,795;20170404,816,793,806,806,5538284,803;20170405,811,801,801,805,4376242,805;20170408,810,785,803,785,5898574,798;20170409,799,776,782,782,3316939,793;20170410,815,785,785,801,10869114,803;20170412,806,789,800,796,5052958,799;20170415,814,785,807,791,8463759,800;20170416,800,786,791,790,5363956,794;20170417,788,779,786,785,5282840,787;20170418,793,774,787,774,7011269,784;20170419,823,775,775,823,14166075,815;20170422,855,831,855,855,58659403,855;20170423,876,842,863,850,16111417,855;20170424,852,813,849,816,15396571,826;20170426,837,814,837,823,3335209,824;20170429,829,803,819,806,4069563,819;20170430,821,800,819,808,7020319,806;20170501,816,793,816,794,4753430,801;20170502,811,785,794,800,5499786,798;20170503,807,793,793,793,8717074,799;20170506,800,780,800,793,5498420,791;20170507,807,779,797,788,4998546,790;20170508,792,777,786,785,7851638,780;20170509,819,776,791,807,13984716,810;20170510,827,793,817,814,5267702,809;20170513,832,785,796,794,10541319,803;20170514,829,793,795,821,14057442,806;20170515,829,797,816,821,27542068,805;20170516,844,814,820,844,32800976,842;20170517,883,847,854,883,53388073,870;20170520,913,887,913,913,73871957,913;20170521,944,908,929,937,54110799,925;20170522,965,901,937,965,44663944,932;20170523,953,927,951,929,44548425,937;20170524,928,908,928,914,6958011,916;20170527,933,901,922,928,9683132,923;20170528,929,908,926,913,5443703,922;20170529,928,908,908,922,10170493,922;20170530,927,914,914,926,9902561,923;20170531,927,914,927,922,6648871,922;20170603,935,914,922,931,7278141,924;20170606,939,925,929,930,5947840,931;20170607,937,898,927,937,35852936,919;20170610,932,916,932,916,3204500,919;20170611,922,911,916,916,12927415,916;20170612,922,905,915,909,1893697,914;20170613,929,897,897,922,4930183,915;20170614,937,916,926,933,8309469,930;20170617,937,909,936,915,4202959,926;20170618,936,906,906,929,15839847,930;20170619,922,911,911,912,10190681,915;20170620,922,906,908,909,1565703,915;20170621,933,897,901,908,5863475,918;20170624,911,901,908,903,497874,917;20170625,922,898,901,901,2118056,914;20170628,935,898,901,908,3466288,917;20170701,929,901,908,908,2522985,918;20170702,929,908,909,908,2949651,919;20170703,915,905,909,906,636672,919;20170704,919,900,904,901,25548329,916;20170705,911,892,900,894,9558804,901;20170708,904,886,900,891,5212835,900;20170709,893,858,887,865,8403336,880;20170710,892,865,888,865,5694334,872;20170711,886,857,870,857,6819316,865;20170712,901,850,857,886,7643474,888;20170715,893,864,875,874,1731143,883;20170716,901,850,865,867,7178981,878;20170717,901,862,872,862,6944482,881;20170718,875,861,862,865,656342,880;20170719,898,861,861,882,3086632,884;20170722,896,866,896,868,1706147,882;20170723,879,857,875,858,7547645,865;20170724,865,852,865,857,2052381,863;20170725,863,848,862,860,1746679,861;20170726,862,843,852,852,2086309,857;20170729,844,817,843,842,3870957,846;20170730,836,804,824,820,10395264,810;20170731,849,803,814,842,6794267,822;20170801,850,822,845,841,2273737,826;20170802,853,819,843,845,2717953,832;20170806,855,829,845,829,1638749,834;20170807,855,824,831,835,2572841,837;20170808,857,840,849,840,1895479,840;20170809,847,824,824,847,1639711,839;20170812,850,834,834,846,14533726,842;20170813,847,834,834,842,2389277,842;20170814,857,830,836,843,6663804,844;20170815,886,830,830,886,55259857,885;20170816,927,886,893,926,15860774,906;20170819,924,901,923,920,10216704,917;20170820,929,901,907,922,11968599,922;20170821,929,879,915,906,2540748,919;20170822,920,891,893,908,6250718,914;20170823,958,891,914,958,59538653,929;20170826,976,922,922,976,25661836,968;20170827,1002,973,978,980,6919049,984;20170828,1009,978,981,1009,10882054,994;20170829,1023,987,1007,1022,29034740,1008;20170830,1045,1007,1026,1043,15217659,1017;20170902,1023,1012,1023,1020,7791970,1017;20170903,1017,984,1017,1009,24430007,1002;20170904,1009,984,995,1003,9651788,1001;20170905,1009,987,1009,1007,16838773,1001;20170906,1009,990,994,990,38728364,1003;20170912,1031,1005,1031,1005,3197750,1013;20170913,1049,1015,1016,1046,6999867,1036;20170916,1063,1046,1046,1054,7671720,1061;20170917,1078,1059,1069,1076,7640524,1071;20170918,1084,1071,1071,1082,18407229,1079;20170919,1099,1075,1080,1099,54333582,1090;20170920,1117,1090,1102,1117,36566819,1105;20170923,1120,1061,1084,1075,4237707,1100;20170924,1080,1049,1072,1079,8080666,1063;20170925,1055,1031,1049,1039,757173,1061;20170926,1075,1044,1046,1069,3319891,1064;20170927,1087,1045,1084,1076,3880322,1069;20171002,1080,1043,1056,1070,4622529,1069;20171003,1069,1016,1069,1061,11849609,1024;20171004,1054,1030,1046,1046,2103394,1031;20171007,1034,1001,1010,1031,1990448,1029;20171008,1075,1024,1034,1031,15196693,1046;20171009,1053,1031,1046,1046,3776553,1046;20171010,1082,1024,1046,1078,5065281,1052;20171011,1072,1052,1060,1061,4303439,1055;20171014,1087,1047,1047,1087,4400396,1072;20171015,1091,1032,1075,1091,5957031,1062;20171016,1091,1043,1065,1091,5713021,1078;20171017,1090,1061,1076,1079,3625285,1078;20171018,1102,1056,1062,1099,16288383,1094;20171021,1124,1094,1094,1121,7936940,1114;20171022,1120,1092,1114,1115,15261314,1114;20171023,1121,1085,1120,1120,2043292,1114;20171024,1122,1106,1121,1114,2207018,1115;20171025,1120,1076,1076,1115,3564050,1115;20171028,1120,1099,1117,1114,3481991,1115;20171029,1122,1076,1115,1115,17058659,1105;20171030,1114,1099,1106,1106,1428324,1105;20171031,1120,1050,1084,1118,9052021,1090;20171101,1114,1078,1078,1114,3284485,1098;20171104,1114,1082,1113,1083,5130456,1103;20171105,1091,1057,1072,1069,499377,1101;20171106,1101,1069,1101,1069,831113,1098;20171107,1076,1062,1064,1071,1243505,1093;20171108,1099,1067,1080,1067,597350,1091;20171111,1091,1060,1091,1060,945913,1090;20171112,1084,1058,1084,1059,850227,1087;20171113,1082,1041,1082,1046,2048077,1079;20171114,1069,1059,1068,1069,752580,1078;20171115,1106,1062,1069,1064,3894480,1084;20171118,1099,1059,1098,1061,362520,1083;20171120,1087,1062,1076,1065,346619,1082;20171121,1076,1031,1076,1046,1582061,1075;20171122,1068,1037,1068,1038,1202497,1071;20171125,1070,1040,1061,1046,1258115,1068;20171126,1084,1039,1061,1039,1725861,1064;20171128,1061,1034,1061,1044,1240700,1061;20171129,1059,1025,1059,1031,2322392,1053;20171202,1105,1046,1099,1060,5718254,1079;20171203,1046,1043,1046,1043,779830,1075;20171204,1046,1035,1043,1035,934137,1071;20171205,1033,1029,1033,1029,468132,1068;20171209,1032,1022,1031,1032,1004297,1062;20171210,1040,1028,1034,1031,1832470,1055;20171211,1046,1017,1032,1018,2254112,1046;20171212,1031,1000,1031,1017,2930250,1033;20171213,1030,1016,1030,1024,3368477,1028;20171216,1079,1046,1046,1079,22811578,1069;20171217,1099,1043,1095,1046,6620587,1069;20171218,1071,1031,1046,1031,2400967,1058;20171219,1058,1016,1058,1023,4650548,1037;20171220,1024,1016,1020,1024,978942,1034;20171223,1036,1013,1031,1023,2088978,1030;20171224,1035,1020,1031,1028,1971208,1029;20171225,1039,1016,1039,1019,2168097,1028;20171226,1033,1017,1017,1031,1604282,1028;20171227,1037,1020,1022,1025,915234,1028;20171230,1020,977,1020,977,3646592,1007;20171231,972,957,972,957,19644278,957;20180101,1004,923,923,968,9357212,963;20180102,1002,951,997,970,3260203,965;20180103,971,917,969,918,25164848,924;20180106,969,924,924,969,13832955,963;20180107,1001,972,975,975,1996142,968;20180108,1009,972,996,998,1667398,974;20180109,1009,975,1004,1009,2956471,986;20180110,1005,978,1005,987,210778,986;20180113,1009,980,980,994,656216,987;20180114,1009,942,1001,965,1801625,986;20180115,1001,973,994,997,2396687,986;20180116,1025,990,999,1002,2716335,994;20180117,1022,1001,1016,1003,1262147,996;20180120,1016,972,1016,980,891219,995;20180121,1009,958,979,958,2358525,987;20180122,1001,957,957,969,1657853,987;20180123,992,961,962,964,858603,985;20180124,978,965,977,966,2008249,981;20180127,987,964,972,966,725183,979;20180128,972,959,966,963,1484308,976;20180129,972,960,962,961,846381,975;20180130,972,958,960,958,967898,973;20180131,972,954,970,972,756707,972;20180203,972,954,972,957,479362,971;20180204,960,931,958,960,998793,968;20180205,990,934,971,941,15944382,939;20180206,972,939,960,971,13416975,948;20180207,995,968,971,995,11959048,991;20180210,1040,1006,1013,1028,13289601,1022;20180212,1047,1010,1024,1046,6748598,1040;20180213,1054,1031,1046,1046,9579742,1046;20180214,1057,1043,1046,1054,10283951,1049;20180217,1064,1049,1049,1061,8807050,1058;20180218,1076,1058,1072,1069,5565644,1066;20180219,1087,1055,1076,1061,6668299,1063;20180221,1089,1060,1089,1084,6685541,1068;20180224,1091,1062,1091,1070,5529561,1069;20180225,1105,1061,1061,1070,7937472,1077;20180226,1084,1061,1077,1061,2669723,1074;20180227,1062,1046,1062,1046,5964327,1053;20180228,1054,1034,1054,1034,12197378,1046;20180303,1046,1036,1039,1036,1575410,1046;20180304,1052,1036,1052,1036,5153006,1045;20180305,1084,1032,1046,1062,5050100,1063;20180306,1076,1031,1076,1031,2540419,1056;20180307,1056,1031,1056,1031,2777475,1051;20180310,1050,1039,1047,1040,3161220,1049;20180311,1053,1032,1053,1032,3070896,1046;20180312,1054,1040,1054,1040,4208695,1048;20180313,1064,1045,1046,1045,7330404,1055;20180314,1061,1044,1061,1046,4722694,1052;20180317,1058,1044,1058,1045,3029710,1052;20180318,1058,1049,1058,1049,2541081,1052;20180319,1061,1046,1061,1046,3016922,1053;20180325,1046,1009,1039,1024,195503,1052;20180326,1031,1009,1031,1010,289821,1051;20180327,1024,1020,1023,1024,95404,1050;20180328,1046,1011,1013,1024,603501,1049;20180403,1024,1016,1016,1024,9756,1049;20180404,1047,1015,1015,1031,1239867,1048;20180407,1015,1001,1009,1013,297945,1046;20180408,1008,994,997,994,1278898,1037;20180409,1028,985,987,985,1220016,1030;20180410,999,987,988,987,428030,1028;20180411,1000,977,978,977,427130,1025;20180415,975,974,975,974,76317,1024;20180416,979,973,974,973,728613,1019;20180417,994,968,968,979,660867,1014;20180418,972,964,972,964,1285034,1005;20180421,984,955,984,964,290217,1003;20180422,957,953,953,953,1254400,995;20180423,966,945,945,945,518735,992;20180424,972,942,945,942,734671,987;20180425,946,938,938,938,727108,983;20180428,938,934,934,938,2052542,969;20180429,926,921,921,921,1438190,960;20180430,927,913,920,927,711241,957;20180501,919,909,913,910,1478927,948;20180505,911,901,901,901,1099722,941;20180506,900,895,896,895,1192579,934;20180507,938,897,916,923,440599,933;20180508,934,900,930,900,298837,933;20180509,979,889,889,948,5297060,947;20180512,972,933,972,964,1070652,948;20180513,971,952,957,957,449624,948;20180514,957,951,957,951,776257,949;20180515,972,949,971,949,662038,949;20180516,960,945,960,945,325956,949;20180519,945,902,945,902,2278392,939;20180520,937,892,924,897,3983043,916;20180521,918,897,897,901,359976,916;20180522,897,881,892,882,604451,914;20180523,901,877,879,889,420167,913;20180526,904,876,876,894,286116,912;20180527,897,880,896,880,527065,910;20180528,897,878,893,882,479997,909;20180529,893,874,878,877,646116,906;20180530,878,867,876,878,2076463,896;20180602,889,860,875,860,1357124,892;20180603,871,856,871,865,920051,889;20180609,874,845,874,856,2786801,875;20180610,877,851,877,851,362303,874;20180611,881,852,856,862,632986,874;20180612,895,856,880,873,1525702,875;20180613,917,877,877,904,2139403,883;20180617,927,912,927,927,8183759,927;20180618,974,972,974,974,114127487,974;20180619,1022,981,988,1022,41701176,1016;20180620,1061,1032,1045,1061,9844267,1052;20180623,1066,999,1031,1024,21812253,1037;20180624,1076,1001,1031,1009,17246633,1026;20180625,1075,1010,1025,1016,46284396,1043;20180626,1016,991,1016,991,6644342,1001;20180627,972,951,960,951,16497133,954;20180630,951,907,929,909,14412273,916;20180701,901,871,899,889,7666163,877;20180702,916,877,890,880,7289592,892;20180703,920,882,903,883,10094227,901;20180704,909,878,907,890,6331150,892;20180707,912,877,897,880,2387449,891;20180708,896,882,896,882,2568863,890;20180710,897,876,878,878,3533440,889;20180711,895,848,875,873,7295874,882;20180714,892,859,892,867,2512433,877;20180715,882,850,874,857,3588495,868;20180716,889,852,859,859,3322587,868;20180717,882,846,857,848,3871197,862;20180718,867,841,846,856,3234921,856;20180721,865,853,860,853,1159128,856;20180722,867,844,866,845,3131501,853;20180723,856,833,848,852,4599249,849;20180724,859,830,838,830,3055711,843;20180725,855,826,851,836,2808943,840;20180728,882,851,852,882,24573144,880;20180729,923,898,923,923,23782759,922;20180730,968,968,968,968,8611424,968;20180731,1016,1016,1016,1016,7873058,1015;20180801,1058,972,1007,1058,39639666,1040;20180804,1063,988,1054,1036,20904257,1022;20180805,1072,1036,1046,1072,27716004,1071;20180806,1124,1072,1124,1091,41359043,1118;20180807,1143,1063,1067,1063,23890548,1089;20180808,1061,1035,1061,1035,7922928,1043;20180811,1016,991,1016,991,2016623,1028;20180812,1061,978,979,998,22748865,1007;20180813,1039,972,994,1027,7805507,1015;20180814,1046,1004,1031,1004,4397873,1025;20180815,1009,986,1001,995,1341971,1020;20180818,1004,975,987,1001,2576135,1010;20180819,1039,997,1009,1009,5365137,1011;20180820,1032,997,1022,1016,6564630,1016;20180821,1014,994,1005,1009,4747065,1006;20180825,1023,990,1015,1001,3517243,1004;20180826,1024,1001,1009,1016,4782829,1010;20180827,1044,1010,1016,1039,6076481,1027;20180828,1070,1007,1045,1022,6280241,1040;20180829,1040,1013,1039,1014,2178467,1034;20180901,1044,983,1044,993,2966791,1021;20180902,1070,987,988,1024,10912866,1038;20180903,1090,1043,1090,1061,9226243,1075;20180908,1171,1101,1118,1108,10153577,1129;20180909,1129,1100,1117,1121,19522224,1115;20180910,1171,1130,1130,1171,16723159,1164;20180911,1222,1176,1222,1183,33640413,1203;20180912,1262,1198,1207,1241,48624851,1248;20180915,1310,1231,1310,1265,79034482,1296;20180916,1303,1231,1262,1266,66849986,1255;20180917,1317,1280,1316,1317,111906610,1315;20180918,1357,1296,1356,1356,88525989,1336;20180922,1403,1357,1379,1403,167735984,1400;20180923,1455,1357,1424,1394,87158702,1418;20180924,1426,1375,1413,1413,60289512,1402;20180925,1472,1414,1424,1472,64377399,1467;20180926,1540,1540,1540,1540,11526224,1540;20180929,1617,1617,1617,1617,22850003,1617;20180930,1697,1674,1697,1696,70663739,1697;20181002,1613,1532,1613,1532,2724840,1611;20181003,1559,1531,1531,1537,106835792,1533;20181006,1609,1457,1609,1457,71451087,1553;20181007,1507,1476,1476,1476,56184164,1477;20181008,1432,1403,1403,1419,40352738,1407;20181009,1476,1339,1403,1381,61275079,1438;20181010,1409,1366,1375,1366,44006234,1370;20181013,1338,1302,1320,1302,7439366,1307;20181014,1242,1242,1242,1242,2688694,1275;20181015,1336,1247,1247,1295,45066249,1293;20181016,1341,1251,1338,1277,41513983,1296;20181017,1271,1231,1240,1233,39797446,1239;20181020,1301,1274,1281,1301,17651495,1296;20181021,1360,1306,1360,1360,56717436,1355;20181022,1420,1326,1420,1363,49211505,1374;20181023,1382,1305,1382,1305,21411959,1317;20181024,1314,1252,1303,1252,11961491,1267;20181027,1234,1204,1212,1204,19848877,1208;20181028,1242,1170,1206,1235,17246493,1212;20181029,1266,1205,1239,1228,17623340,1222;20181031,1241,1191,1241,1213,16777079,1208;20181103,1220,1187,1214,1206,13212231,1201;20181104,1261,1192,1192,1261,30144741,1234;20181105,1295,1295,1295,1295,6520120,1295;20181106,1359,1326,1359,1359,72994955,1354;20181110,1372,1293,1372,1293,19798441,1311;20181111,1345,1266,1318,1289,22179377,1301;20181112,1341,1304,1323,1319,29303257,1326;20181113,1356,1301,1343,1319,34702939,1331;20181114,1397,1304,1397,1323,39163179,1363;20181117,1334,1296,1311,1296,5762380,1302;20181118,1270,1237,1251,1237,16883325,1238;20181119,1265,1206,1210,1214,8345367,1231;20181120,1244,1185,1213,1215,12984923,1204;20181121,1219,1186,1206,1209,9367011,1203;20181124,1205,1147,1199,1161,15408847,1164;20181126,1167,1106,1153,1106,9604440,1121;20181127,1153,1110,1130,1130,6960244,1127;20181128,1138,1072,1138,1077,23027188,1075;20181201,1070,1022,1056,1022,42997233,1024;20181202,1069,1032,1032,1048,13008765,1051;20181203,1103,1063,1073,1103,27616167,1100;20181204,1123,1056,1123,1064,8910068,1078;20181205,1074,1055,1062,1065,8843870,1061;20181208,1088,1055,1088,1065,7619441,1071;20181209,1070,1040,1070,1051,8768034,1049;20181210,1066,1029,1039,1044,7781856,1044;20181211,1063,1032,1046,1054,5292721,1048;20181212,1058,1032,1055,1040,7386068,1045;20181215,1046,1002,1045,1017,7252203,1024;20181216,1017,984,1008,999,6424614,1002;20181217,1010,985,995,1000,6982299,993;20181218,1007,984,1001,993,5551888,990;20181219,993,953,990,971,7633221,965;20181222,969,934,969,941,6944796,945;20181223,992,965,965,992,5340464,975;20181224,1023,996,1023,1017,15836912,1015;20181225,1008,982,1002,987,6504219,996;20181226,1044,994,994,1036,12761190,1028;20181229,1072,1005,1045,1042,12064889,1045;20181230,1050,1025,1043,1029,6937006,1032;20181231,1032,1012,1032,1024,5709327,1023;20190101,1027,999,1017,1002,10170983,1013;20190102,1017,988,1013,1000,5361048,1002;20190105,1019,1004,1017,1011,6433754,1011;20190106,1062,987,1020,1062,20381306,1040;20190107,1092,1069,1078,1092,15138399,1088;20190108,1141,1094,1116,1108,28267193,1115;20190114,1123,1064,1123,1067,4894323,1089;20190115,1100,1063,1078,1091,33742562,1072;20190116,1126,1083,1100,1123,44506541,1121;20190119,1167,1114,1138,1123,12010882,1141;20190120,1146,1084,1130,1085,10267490,1110;20190121,1109,1055,1081,1055,13877191,1056;20190122,1078,1044,1054,1059,8880183,1057;20190123,1077,1052,1055,1055,7586767,1056;20190126,1070,1040,1057,1055,4921885,1056;20190127,1109,1056,1056,1109,16839747,1103;20190128,1144,1108,1115,1108,17292265,1128;20190129,1138,1072,1105,1093,8243553,1106;20190130,1100,1063,1097,1089,5899053,1084;20190202,1097,1058,1093,1058,4514504,1080;20190203,1088,1026,1070,1054,10349300,1045;20190204,1070,1048,1063,1054,6904534,1058;20190205,1070,1055,1063,1066,4812163,1059;20190206,1075,1052,1070,1064,6116666,1058;20190210,1087,1059,1059,1081,3460026,1066;20190212,1087,1061,1070,1066,9299885,1065;20190213,1069,1040,1063,1040,5518734,1055;20190216,1064,1040,1062,1056,5942166,1052;20190217,1068,1017,1062,1017,9505842,1043;20190218,1088,1027,1027,1080,14798080,1060;20190219,1108,1063,1084,1065,5173571,1081;20190220,1092,1057,1092,1078,5511205,1073;20190223,1127,1079,1082,1106,14987701,1117;20190224,1122,1081,1116,1105,6076606,1106;20190225,1139,1101,1115,1131,24820876,1127;20190226,1182,1094,1103,1182,51276722,1175;20190227,1233,1202,1219,1233,81241053,1228;20190302,1265,1207,1251,1208,23467617,1228;20190303,1289,1208,1216,1289,52593373,1273;20190304,1311,1247,1281,1253,18379841,1280;20190305,1275,1223,1259,1223,14491401,1240;20190306,1245,1198,1213,1221,10819037,1222;20190309,1282,1223,1236,1280,50011291,1271;20190310,1334,1300,1304,1311,58401306,1320;20190311,1349,1282,1323,1283,31732957,1311;20190312,1310,1274,1286,1280,32078247,1296;20190313,1305,1251,1272,1262,21790374,1276;20190316,1306,1258,1280,1268,11241336,1276;20190317,1281,1243,1259,1260,17196708,1257;20190318,1289,1248,1259,1266,13851345,1271;20190319,1288,1252,1267,1277,21258178,1274;20190325,1311,1271,1311,1293,2874543,1277;20190326,1307,1275,1277,1280,20922163,1289;20190327,1282,1256,1281,1266,10533631,1265;20190330,1283,1261,1266,1274,15130179,1271;20190331,1304,1253,1285,1273,9322062,1271;20190406,1288,1262,1273,1279,12724077,1271;20190407,1334,1269,1272,1334,247171317,1326;20190408,1393,1393,1393,1393,174628313,1393;20190409,1462,1393,1417,1403,180621619,1432;20190410,1504,1406,1417,1504,139662304,1488;20190413,1549,1473,1504,1482,49858856,1504;20190414,1566,1451,1483,1556,76221597,1528;20190415,1604,1560,1574,1580,73196572,1598;20190416,1636,1518,1568,1518,53839921,1565;20190417,1559,1489,1515,1522,35274627,1519;20190420,1575,1529,1553,1554,31693314,1549;20190422,1575,1472,1575,1472,66887537,1491;20190423,1471,1418,1444,1418,42238537,1437;20190424,1461,1387,1418,1432,38083403,1414;20190427,1455,1409,1425,1417,20661182,1424;20190428,1477,1424,1447,1459,24598651,1454;20190429,1507,1444,1467,1485,60800903,1469;20190430,1536,1449,1514,1466,39046714,1482;20190501,1551,1447,1463,1511,63187852,1506;20190504,1580,1545,1560,1580,46811962,1579;20190505,1654,1522,1643,1547,75452340,1591;20190506,1560,1512,1557,1512,31418368,1521;20190507,1477,1445,1447,1445,12153304,1447;20190508,1505,1394,1394,1433,38087875,1434;20190511,1455,1363,1432,1363,37634536,1384;20190512,1379,1315,1335,1319,44154871,1317;20190513,1382,1293,1319,1382,20111518,1345;20190514,1412,1346,1409,1379,32518141,1390;20190515,1409,1341,1409,1372,28330092,1375;20190518,1403,1349,1364,1357,19653180,1384;20190519,1378,1341,1361,1364,11027562,1360;20190520,1366,1327,1366,1351,10429761,1351;20190521,1393,1357,1357,1368,11341425,1366;20190522,1396,1362,1372,1393,19672908,1384;20190525,1409,1387,1403,1393,8722120,1393;20190526,1422,1369,1409,1374,14900447,1387;20190528,1408,1364,1393,1390,29959629,1384;20190529,1413,1387,1396,1391,23194106,1398;20190601,1383,1341,1383,1364,12553737,1358;20190602,1390,1345,1364,1379,21526281,1364;20190603,1432,1372,1390,1432,26886789,1421;20190608,1492,1402,1437,1485,249837807,1485;20190609,1513,1455,1507,1462,64407287,1484;20190610,1552,1458,1483,1492,92143690,1513;20190611,1548,1473,1511,1511,83857695,1516;20190612,1591,1510,1510,1591,267916721,1589;20190615,1668,1575,1622,1668,376280646,1659;20190616,1742,1678,1693,1693,210846735,1733;20190617,1809,1658,1711,1791,95035404,1762;20190618,1849,1734,1801,1746,105343621,1810;20190619,1764,1720,1758,1720,42375793,1727;20190622,1688,1641,1641,1675,99257333,1648;20190623,1730,1681,1689,1730,85384794,1722;20190624,1805,1723,1805,1771,96198826,1770;20190625,1774,1688,1770,1733,40505556,1721;20190626,1764,1650,1733,1650,58022101,1689;20190630,1732,1615,1696,1626,55974734,1661;20190701,1687,1610,1656,1636,52623392,1644;20190702,1726,1658,1658,1726,37514006,1704;20190703,1789,1738,1770,1771,79622702,1776;20190706,1783,1687,1779,1703,34157845,1716;20190707,1741,1666,1702,1711,29022737,1710;20190708,1759,1690,1733,1696,41601980,1720;20190713,1734,1672,1733,1688,12821855,1697;20190714,1702,1649,1702,1652,34212112,1672;20190715,1666,1620,1666,1623,42129828,1642;20190716,1633,1561,1633,1561,108888579,1562;20190717,1635,1532,1532,1588,38361902,1586;20190720,1666,1583,1666,1620,30146337,1622;20190721,1688,1620,1632,1620,29473404,1663;20190722,1701,1628,1666,1643,28575040,1658;20190723,1681,1601,1666,1659,21645056,1640;20190724,1712,1651,1659,1682,28087193,1683;20190727,1722,1652,1718,1682,17797150,1689;20190728,1686,1617,1684,1679,26727055,1647;20190729,1703,1645,1688,1646,18548216,1666;20190730,1671,1635,1669,1650,13056913,1653;20190731,1735,1643,1666,1735,48868236,1710;20190803,1795,1748,1794,1790,95746656,1792;20190804,1794,1724,1790,1737,26210989,1748;20190805,1817,1732,1741,1809,55131577,1767;20190806,1855,1784,1822,1846,71639680,1831;20190807,1922,1835,1850,1922,151201409,1907;20190813,1991,1913,1991,1946,27182649,1951;20190814,2048,1947,1948,2048,36529748,2033;20190817,2114,2027,2051,2043,48222869,2065;20190818,2125,2004,2011,2004,31709546,2041;20190819,2115,2011,2027,2083,49630989,2060;20190821,2154,2044,2083,2069,38347132,2091;20190824,2101,2016,2067,2051,43393286,2052;20190825,2121,1999,2043,2019,49043784,2037;20190827,2214,2091,2165,2097,76171744,2156;20190828,2165,2075,2109,2161,40914964,2144;20190831,2225,2116,2164,2123,38230118,2166;20190901,2173,2086,2129,2116,24401457,2129;20190902,2137,2024,2118,2042,41037656,2049;20190903,2081,2012,2012,2019,13961114,2043;20190904,2097,2031,2042,2051,15639907,2064;20190908,2162,2075,2162,2115,27964882,2119;20190911,2147,2082,2147,2083,29416783,2104;20190914,2153,2080,2105,2085,40623992,2100;20190915,2146,2059,2091,2091,28389551,2102;20190916,2139,2075,2133,2083,57258662,2101;20190917,2083,1996,2083,1996,36647240,2024;20190918,2041,1931,2003,1999,30622901,1993;20190921,2071,1952,1952,2011,13271662,2016;20190922,2116,1999,2043,2116,109153167,2047;20190923,2149,2124,2146,2149,68493747,2149;20190924,2219,2083,2195,2145,51480612,2152;20190925,2151,2099,2151,2135,26768991,2123;20190928,2183,2118,2155,2141,48974484,2146;20190929,2162,2083,2155,2139,33565671,2122;20190930,2228,2175,2179,2228,312945326,2227;20191001,2322,2167,2322,2203,123739075,2278;20191002,2243,2171,2243,2202,53869334,2197;20191005,2258,2097,2234,2115,70550767,2165;20191006,2137,2059,2137,2091,26997589,2095;20191007,2171,2058,2146,2067,29566046,2097;20191008,2111,1992,2111,1992,23932107,2021;20191009,2056,1936,1964,2051,26764208,1999;20191012,2098,2027,2081,2098,30897551,2083;20191013,2187,2059,2155,2067,31365703,2127;20191014,2129,2021,2129,2039,14396052,2045;20191015,2083,1943,2019,1943,24600008,1956;20191016,1978,1859,1899,1859,35177133,1893;20191020,1923,1799,1863,1809,55072511,1807;20191021,1875,1746,1747,1859,28226875,1807;20191022,1897,1819,1883,1864,23725595,1874;20191023,1899,1803,1883,1874,13409524,1838;20191026,1888,1811,1880,1882,7601820,1839;20191028,1915,1833,1843,1895,10676243,1881;20191030,1947,1908,1920,1938,9738683,1934;20191102,1967,1867,1963,1883,12359454,1909;20191103,1931,1843,1881,1871,12589082,1882;20191104,1906,1819,1906,1831,11435286,1842;20191105,1875,1819,1871,1871,11137106,1845;20191109,1901,1831,1884,1848,11786253,1846;20191110,1859,1820,1859,1847,8087259,1834;20191111,1848,1820,1824,1841,11122615,1833;20191112,1887,1827,1835,1843,15195538,1847;20191113,1847,1830,1847,1835,16931580,1838;20191116,1896,1813,1895,1819,10724962,1850;20191117,1819,1758,1765,1759,27322143,1766;20191118,1817,1722,1722,1775,6934684,1781;20191119,1807,1776,1807,1799,17041077,1788;20191120,1857,1723,1812,1841,13939625,1806;20191123,1896,1837,1857,1896,53335808,1883;20191124,1922,1863,1920,1876,21004802,1888;20191125,1914,1845,1886,1888,16459712,1883;20191126,1905,1848,1905,1851,15452986,1861;20191127,1882,1843,1863,1873,15919861,1852;20191130,1891,1844,1883,1874,22241232,1861;20191201,1954,1867,1874,1954,52419281,1924;20191202,1962,1891,1955,1920,35989085,1935;20191203,1955,1916,1953,1924,23895677,1932;20191208,2123,2027,2028,2083,85111307,2061;20191209,2141,2069,2129,2099,34017782,2113;20191210,2131,2075,2123,2106,37434734,2107;20191211,2163,2059,2108,2076,24327172,2099;20191214,2098,2069,2082,2097,36899740,2085;20191215,2133,2044,2093,2081,27011728,2086;20191216,2115,2059,2075,2087,29019011,2081;20191221,2147,2099,2118,2118,41406237,2126;20191222,2232,2179,2232,2232,145207655,2223;20191223,2314,2179,2278,2266,106093389,2266;20191224,2371,2235,2274,2266,93742344,2339;20191225,2379,2292,2363,2357,46838292,2343;20191228,2460,2395,2403,2460,136119450,2456;20191229,2523,2403,2500,2481,60880531,2472;20191230,2554,2422,2472,2493,57061128,2482;20191231,2548,2399,2500,2473,53508278,2450;20200101,2539,2445,2456,2516,52303948,2506;20200104,2381,2381,2381,2381,9388159,2381;20200105,2353,2262,2262,2275,142053931,2276;20200107,2313,2162,2242,2162,40272598,2195;20200108,2152,2152,2152,2152,9903819,2152;20200111,2194,2194,2194,2194,3728553,2194;20200112,2303,2303,2303,2303,4291353,2303;20200113,2418,2418,2418,2418,49823157,2418;20200114,2538,2419,2492,2538,118242306,2513;20200115,2590,2476,2539,2572,59614192,2559;20200118,2686,2589,2652,2686,71677287,2678;20200121,2827,2686,2739,2709,78310332,2747;20200122,2788,2644,2737,2725,60298482,2720;20200125,2843,2708,2709,2841,67093694,2782;20200126,2921,2782,2895,2921,110141555,2909;20200127,3054,2850,2946,3028,64936639,3016;20200128,3145,2980,3100,3017,49052601,3078;20200201,3036,2924,3036,2924,65938438,2948;20200202,2957,2869,2872,2941,65283944,2897;20200203,3008,2953,2964,2988,33326354,2982;20200204,3060,2846,3060,2881,52730685,2911;20200205,2992,2900,2924,2940,40332188,2938;20200208,3084,3000,3084,3068,155753935,3075;20200209,3181,3061,3181,3075,114170853,3107;20200210,3125,2952,3124,3023,68106221,3016;20200212,3159,3045,3045,3092,72323793,3117;20200215,3273,3124,3273,3205,73250377,3201;20200216,3351,3253,3273,3338,64268131,3314;20200217,3413,3215,3413,3259,47313381,3290;20200218,3245,3126,3245,3183,35972014,3169;20200219,3227,3086,3227,3139,56553865,3151;20200222,3151,2994,2994,3081,55214529,3074;20200223,3227,3124,3124,3227,53601646,3221;20200224,3348,3229,3269,3338,55258909,3281;20200225,3425,3249,3331,3397,44841766,3332;20200226,3491,3302,3491,3325,46911834,3374;20200229,3284,3206,3217,3206,80306467,3211;20200301,3319,3205,3236,3245,61038600,3256;20200302,3381,3256,3285,3333,59246008,3306;20200303,3471,3364,3413,3471,67577246,3439;20200304,3608,3493,3558,3604,49626674,3570;20200307,3688,3406,3597,3445,43741851,3512;20200309,3337,3337,3337,3337,28593425,3337;20200310,3268,3170,3170,3170,46653203,3185;20200311,3282,3193,3193,3203,110389641,3238;20200314,3164,3076,3164,3076,14032356,3093;20200315,3040,2939,2939,2972,68732779,2956;20200316,3104,3044,3044,3104,56347736,3102;20200317,3140,2984,3140,3123,30432576,3092;20200318,3245,3145,3149,3245,62649786,3239;20200324,3293,3077,3253,3077,65724030,3095;20200325,3244,3100,3123,3201,38328778,3181;20200328,3295,3085,3229,3261,53761027,3218;20200329,3318,3165,3313,3205,61933774,3241;20200330,3238,3124,3124,3217,79292065,3181;20200404,3340,3295,3340,3340,149519865,3340;20200405,3507,3507,3507,3507,179213462,3507;20200406,3681,3507,3681,3681,400637669,3680;20200407,3863,3765,3861,3863,142612807,3858;20200408,4051,4051,4051,4051,63137134,4051;20200411,4252,4167,4252,4252,312846084,4252;20200412,4446,4167,4326,4232,294716716,4289;20200413,4294,4075,4294,4075,162696180,4146;20200414,4353,4023,4075,4353,129585778,4248;20200415,4461,4461,4461,4461,17297770,4461;20200418,4684,4684,4684,4684,7753710,4653;20200420,5129,4885,4885,5129,267145418,5127;20200421,5382,4969,5239,5382,227087433,5320;20200422,5586,5388,5575,5586,247034045,5569;20200425,5847,5846,5847,5847,188990695,5847;20200426,6138,5849,6138,6138,272186057,6118;20200427,6423,6139,6304,6423,187168900,6401;20200428,6657,6129,6646,6441,260474483,6413;20200429,6734,6281,6489,6734,198703536,6666;20200502,7000,7000,7000,7000,131426980,7000;20200503,7349,7347,7349,7349,149147613,7349;20200504,8011,6982,7611,7771,301914068,7572;20200505,7915,7194,7915,7194,246817232,7412;20200506,7395,7042,7042,7153,217537282,7142;20200509,7499,7186,7450,7499,181754359,7487;20200510,7861,7860,7861,7861,260148491,7861;20200511,8253,7971,8253,8253,303023086,8243;20200512,8412,7831,8412,7831,196952351,7939;20200513,8318,7542,7542,7542,275553760,7810;20200516,7807,7420,7571,7420,33328075,7496;20200517,7270,7121,7121,7121,138569622,7129;20200518,7485,6930,7234,7250,163041831,7243;20200519,7571,6881,7363,6883,149511223,7098;20200520,7010,6743,6770,6743,45722088,6773;20200523,6435,6114,6435,6329,98493030,6264;20200526,6330,6170,6170,6170,6492684,6187;20200719,12991,12374,12374,12991,74208321,12531;20200720,13158,13158,13158,13158,16956694,13158;20200721,13814,13814,13814,13814,8195116,13696;20200722,14372,14372,14372,14372,8768861,14293;20200725,15008,15008,15008,15008,4234488,14665;20200726,15390,15390,15390,15390,3636792,14988;20200727,15732,15732,15732,15732,30095946,15732;20200728,16516,16516,16516,16516,53390852,16516;20200729,17338,17338,17338,17338,17824521,17338;20200801,18200,18200,18200,18200,5979283,18102;20200802,19002,18200,18200,19002,9624127,19002;20200803,19952,19090,19952,19776,290605688,19942;20200804,20931,18953,19365,19355,406917111,19707;20200805,20686,18728,19384,19384,281168997,18953;20200809,19893,18650,19580,19893,126602992,19443;20200810,18895,17955,18895,18014,103203615,18317;20200811,17622,17407,17407,17407,47875860,17407;20200812,18268,16545,18111,16545,135551437,17328;20200815,16467,16467,16467,16467,14123618,16467;20200816,15644,15644,15644,15644,10056241,15644;20200817,15860,14871,14871,14871,178496237,15067;20200818,15811,15077,15811,15507,111908027,15635;20200819,15360,14861,14881,14861,18404172,14881;20200822,15615,14137,15116,14137,70938471,14411;20200823,14391,13696,13706,13696,21961354,13726;20200824,13353,13040,13040,13040,33209910,13050;20200916,13700,13700,13700,13700,19368913,13700;20200919,14380,14380,14380,14380,25447690,14380;20200920,15090,14800,15090,15090,445712405,15090;20200921,15800,14340,14980,15680,342958075,14970;20200922,15710,15080,15710,15710,180951658,15700;20200923,16480,16480,16480,16480,45274943,16480;20200926,17300,17300,17300,17300,24180929,17300;20200927,18160,16520,18160,18100,481148873,17620;20200928,16740,16740,16740,16740,22810116,16740;20200929,17570,15910,15910,17570,338034969,16930;20200930,17770,15910,17560,17770,142136957,17680;20201003,18560,18560,18560,18560,17339163,18560;20201004,19480,17650,19480,18040,424926978,18920;20201005,18700,17980,17980,17980,134425997,18030;20201006,18800,17130,17700,17910,248358431,17640;20201007,18250,16800,17700,17050,177405862,17300;20201010,17200,16440,16830,16440,161805774,16580;20201011,17400,16600,16600,16950,205015494,17250;20201012,17400,16390,16900,16500,130271265,16540;20201013,16510,15720,16500,15980,135539284,15940;20201014,15900,15150,15900,15150,130500809,15240;20201018,15150,14480,14720,14480,43407233,14540;20201019,13820,13820,13820,13820,17726190,13820;20201020,14290,13130,13130,14050,112069309,13380;20201021,14040,13600,14040,14040,209879933,14020;20201024,14090,13320,13490,13320,59888553,13390;20201026,13440,12730,12730,12850,116656134,12820;20201027,13180,12180,12640,12180,68767607,12350;20201028,13180,11740,11870,12380,140260328,12070;20201031,12670,12070,12130,12670,96520195,12590;20201101,12800,11970,12800,11970,71566524,12150;20201102,12140,11550,11570,11590,79308566,11640;20201104,12220,11660,11970,12220,77823050,12190;20201111,12790,12330,12650,12790,91586063,12690;20201114,13320,12600,12970,13320,55560448,13200;20201115,13860,12540,13860,12700,205649677,13110;20201116,13490,12500,12500,12900,105815738,13080;20201117,13620,12700,12910,13500,138307587,13260;20201118,13920,13330,13600,13920,119680138,13780;20201121,14170,13100,14150,13100,131973043,13570;20201122,13460,12900,13000,12900,104241113,13030;20201123,13490,13000,13010,13100,75623418,13180;20201124,13180,12530,13180,12530,74285456,12680;20201125,12590,12050,12200,12430,134017093,12290;20201128,12900,12020,12050,12900,107759973,12750;20201129,13380,13110,13380,13380,102117384,13370;20201130,14030,13500,13920,14030,133021054,14000;20201201,14510,13500,13990,14110,177935503,14100;20201202,14300,13450,14110,13460,114519838,13830;20201205,13980,13200,13480,13620,101897859,13630;20201206,14100,13700,13800,13840,109776583,13920;20201207,13960,13230,13950,13230,97639943,13550;20201208,13540,12880,13450,13030,118539270,13030;20201209,13680,12940,13110,13680,107276400,13350;20201212,14010,13500,13900,13700,133146976,13890;20201213,13920,13200,13750,13310,99375107,13440;20201214,13710,12770,13400,12770,88112109,12990;20201215,12700,12350,12350,12350,28504542,12360;20201216,12120,11750,11750,12020,106974520,11860;20201219,12450,11910,12290,12440,77375730,12340;20201220,12600,11770,12500,11910,64375332,12090;20201221,12550,11780,12020,12090,72900884,12170;20201222,12460,11840,12400,12210,73624386,12160;20201223,12760,12230,12450,12760,75883435,12700;20201226,13130,12130,13130,12220,87740347,12450;20201227,12690,11880,12180,12170,77338854,12130;20201228,12610,11980,12060,12210,62688383,12250;20201229,12350,11650,12350,11740,67326621,11800;20201230,11720,11210,11650,11270,82965922,11350;20210102,11270,10790,11270,10820,86338070,10820;20210103,11140,10360,10360,10720,55478456,10720;20210104,11200,10700,11000,10970,42871278,10950;20210105,11000,10410,11000,10420,62443948,10500;20210106,10750,9980,10310,10530,120666038,10370;20210109,10880,10260,10700,10870,70593414,10570;20210110,11090,10810,10900,11090,65140832,11060;20210111,11450,10710,11450,10740,57832442,11020;20210112,10740,10470,10580,10470,31076672,10480;20210113,10380,9960,9990,9960,69860430,10000;20210116,9900,9500,9600,9500,30139190,9540;20210118,9150,9070,9070,9070,23964112,9070;20210119,9520,8620,8620,9390,81384277,9130;20210120,9580,9320,9550,9580,21868330,9570;20210123,10040,10040,10040,10040,22603893,10040;20210124,10540,9640,10540,9930,106352223,10210;20210125,10720,9710,9870,10700,108839953,10540;20210126,10850,10020,10850,10020,67659295,10270;20210127,10580,9770,9910,10460,48059448,10280;20210130,10790,10410,10460,10790,44366109,10750;20210131,11190,10220,11000,10220,69313667,10570;20210201,10660,10050,10130,10050,59990221,10150;20210202,10040,9650,9760,9650,61789934,9710;20210203,9950,9230,9700,9230,58600873,9320;20210208,9780,9610,9640,9780,19285906,9730;20210209,10210,9640,9970,10210,80422102,10110;20210213,10710,10410,10600,10710,68091274,10610;20210214,10890,10400,10850,10400,46907054,10550;20210215,10600,10340,10340,10340,35415998,10360;20210216,10300,10160,10170,10160,23281480,10160;20210217,10230,9960,10160,9960,37280495,9980;20210220,10000,9790,9790,9900,22648556,9800;20210221,9790,9610,9620,9610,23909222,9610;20210222,9580,9420,9420,9420,22024850,9420;20210223,9850,9240,9240,9700,44636164,9440;20210224,9800,9260,9790,9280,33768778,9390;20210227,9280,9210,9230,9210,20102063,9210;20210228,9150,9030,9050,9030,10977730,9040;20210301,9200,8860,8860,8980,46944188,8900;20210302,9260,8920,9170,9050,27251200,9070;20210303,9180,8890,8980,8910,19275288,8950;20210306,9290,8920,8940,9180,38278098,9120;20210307,9340,9110,9170,9220,50003980,9210;20210308,9660,9200,9250,9570,34308935,9450;20210309,9760,9300,9670,9330,26378236,9520;20210310,9470,9330,9330,9330,21299775,9340'''
historyData = pd.read_csv(StringIO(historyDataCSV), header=None
                          , lineterminator=';'
                          , index_col=0
                          , names=['Date', 'High', 'Low', 'Open', 'Last', 'Vol', 'Close'])
historyData = historyData.iloc[::-1]
# historyData['yClosePrice'] = historyData['Close'].shift(-1)
historyData['PriceUp'] = np.where(historyData['Close'].shift(-1) < historyData['Close'], 1, 0)

historyData

,High,Low,Open,Last,Vol,Close,PriceUp
Date,,,,,,,
20210310,9470,9330,9330,9330,21299775,9340,0
20210309,9760,9300,9670,9330,26378236,9520,1
20210308,9660,9200,9250,9570,34308935,9450,1
20210307,9340,9110,9170,9220,50003980,9210,1
20210306,9290,8920,8940,9180,38278098,9120,1
...,...,...,...,...,...,...,...
20160501,1072,1053,1072,1053,5008436,1065,0
20160430,1095,1060,1063,1067,7180996,1072,0
20160427,1109,1060,1105,1073,5582589,1079,0


In [48]:
from sklearn.model_selection import  train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import classification_report

In [44]:
# historyData.drop(columns=['Date'])
X = historyData.iloc[:,0:historyData.shape[1]-1].values
Y = historyData.iloc[:,historyData.shape[1]-1].values

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size= 0.2)
tree = DecisionTreeClassifier().fit(x_train, y_train)
print(tree.score(x_test, y_test))


0.6529680365296804


In [47]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size= 0.2, random_state=42)
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.fit_transform(x_test)


x_train

array([[-0.48697863, -0.48763189, -0.48827139, -0.48809767, -0.56735941,
        -0.48688843],
       [-0.47619543, -0.47263643, -0.47534406, -0.47461223, -0.49574528,
        -0.47533655],
       [-0.51299897, -0.51073963, -0.51340789, -0.51290124, -0.51749558,
        -0.51288018],
       ...,
       [ 2.97091344,  3.14569267,  3.0473556 ,  3.06868688, -0.42597039,
         3.06555722],
       [-0.45720762, -0.45567436, -0.45882579, -0.45390245, -0.12899164,
        -0.45824938],
       [-0.50198135, -0.50262734, -0.50167752, -0.50447284, -0.52764493,
        -0.50229095]])

In [49]:
rfc = RandomForestClassifier(n_estimators=200)
rfc.fit(x_train, y_train)
pred_rfc = rfc.predict(x_test)
print(classification_report(y_test, pred_rfc))

              precision    recall  f1-score   support

           0       0.70      0.75      0.72       124
           1       0.64      0.58      0.61        95

    accuracy                           0.68       219
   macro avg       0.67      0.66      0.67       219
weighted avg       0.67      0.68      0.67       219

